## Imports and Prep

In [2]:
import pandas as pd
import os
import re
import os
from PIL import Image
import shutil

In [ ]:
source_folder = 'image_dump'
def clean_invalid_images(folder_path):
    total = 0
    deleted = 0

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        total += 1

        try:
            # Try opening the image
            with Image.open(file_path) as img:
                img.verify()  # Verify image integrity (does not load into memory fully)
        except Exception as e:
            print(f"Deleting invalid image: {filename} ({e})")
            os.remove(file_path)
            deleted += 1

    print(f"Checked {total} images. Deleted {deleted} invalid images.")

def check_image_exists(df, source_folder, image_column='image_path'):
    # Ensure the source folder exists
    if not os.path.exists(source_folder):
        print(f"The source folder {source_folder} does not exist!")
        return df  # Return the dataframe as is without modifications
    
    # Apply function to check image existence
    df['image_exists'] = df[image_column].apply(
        lambda image_filename: os.path.exists(os.path.join(source_folder, image_filename)) if image_filename else False
    )
    
    return df



In [ ]:
final_cols = ["domain", "title", "text", "publish_date", "image_path", "score", "label"]

## Fakeeddit

In [16]:
fakeddit_df = pd.read_csv("../data/fakeddit/fakeddit_sample.csv")

In [18]:
display(fakeddit_df)
print(fakeddit_df.columns)
fakeddit_df['2_way_label'].value_counts()

,Unnamed: 0.1,Unnamed: 0,index,author,clean_title,created_utc,domain,hasImage,id,image_url,...,article_name,url,title_y,text,publish_date,authors,top_image,images,source,image_path
0,0,5576,12354,residentchubbychaser,there are no good options in syria sighs man w...,1492109533,theonion.com,True,657h33,https://external-preview.redd.it/QGOLuH4_e7M5f...,...,usanews_theonion_scraped_articles_12354.json,https://www.texastribune.org/2016/10/28/uttt-p...,UT/TT Poll: Texas voters feeling high anxiety ...,Voters in the party that has not lost a statew...,2016-10-28,"['Ross Ramsey', 'Oct.', 'Am Central']",https://thumbnails.texastribune.org/YS7n_7vGhg...,['https://thumbnails.texastribune.org/YS7n_7vG...,https://www.texastribune.org,image_dump\TT-RossCharts_FRI-01.jpeg
1,1,3962,8782,dwaxe,not very scary honestly this little girl is po...,1568059431,clickhole.com,True,d1w7ur,https://external-preview.redd.it/nkiyXncogSDx5...,...,usanews_theonion_scraped_articles_8782.json,http://www.nj.com/politics/index.ssf/2017/11/m...,Phil Murphy beats Kim Guadagno to succeed Chri...,"TRENTON -- Phil Murphy, a former Wall Street e...",2017-11-08,"['Brent Johnson', 'Bjohnson Njadvancemedia.Com...",http://www.nj.com/pf/resources/images/nj/favic...,['http://www.nj.com/pf/resources/images/common...,http://www.nj.com,image_dump\favicon.ico
2,2,5841,12927,residentchubbychaser,god deploys more mosquitoes to us,1497908290,theonion.com,True,6i9l51,https://external-preview.redd.it/mjKGy8mJI-JO4...,...,usanews_theonion_scraped_articles_12927.json,https://www.clickhole.com/devastating-blow-to-...,Devastating Blow To Feminism: This Woman Sucks,Devastating Blow To Feminism: This Woman Sucks...,2019-06-12,"['Clickhole Editorial', 'June']",https://clickhole.com/wp-content/uploads/2019/...,['https://clickhole.com/wp-content/uploads/202...,https://www.clickhole.com,image_dump\sng0fvtlhsq8btayt79t.jpg
3,3,357,801,dwaxe,gordon ramsay said what,1553191320,clickhole.com,True,b3tree,https://external-preview.redd.it/5W3TOi9qCR1Ko...,...,usanews_theonion_scraped_articles_801.json,http://www.theverge.com/2017/1/29/14430082/fou...,"In the last 24 hours, four federal courts have...","Last night, a Federal District court issued an...",2017-01-29,['Andrew Liptak'],https://platform.theverge.com/wp-content/uploa...,['https://platform.theverge.com/wp-content/upl...,http://www.theverge.com,image_dump\DSC_0178.0.jpg
4,4,1804,4087,dwaxe,blog i wont let my son play wheelchair basketb...,1505053996,clickhole.com,True,6z8j5j,https://external-preview.redd.it/vJiR9pAFoQSTV...,...,usanews_theonion_scraped_articles_4087.json,http://www.foxnews.com/health/2016/11/04/healt...,Health officials find first cases of new super...,Just five months after federal health official...,2016-11-04,[],https://static.foxnews.com/foxnews.com/content...,['https://a57.foxnews.com/cf-images.us-east-1....,http://www.foxnews.com,image_dump\candida-auris_2016.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7418,8082,21816,32140,Seitz_,brexit mps vote by to to seek delay to eu depa...,1552590210,bbc.co.uk,True,b14akb,https://external-preview.redd.it/C3jff__SjprvR...,...,neutralnews_nottheonion_scraped_articles_32140...,http://www.bbc.co.uk/news/uk-politics-47576813,Brexit: MPs vote by a majority of 211 to seek ...,MPs have voted by 413 to 202 - a majority of 2...,NaN,[],https://ichef.bbci.co.uk/ace/branded_news/1200...,['https://ichef.bbci.co.uk/ace/standard/385/cp...,http://www.bbc.co.uk,image_dump\_106029989_ukparliament_jessicatayl...
7419,8083,1622,2257,limahart,field hockey team rallies around teammate with...,1474724364,boston.cbslocal.com,True,54a5kh,https://external-preview.redd.it/VYkVPVWBJfGEc...,...,upliftingnews_usnews_scraped_articles_2257.json,http://boston.cbslocal.com/2016/09/23/tewksbur...,Field Hockey Team Rallies Around Player Living...,TEWKSBURY (CBS) - It's a story that plays out ...,2016-09-23,[],https://assets1.cbsnewsstatic.com/hub/i/r/2015...,['https:/

Index(['Unnamed: 0.1', 'Unnamed: 0', 'index', 'author', 'clean_title',
       'created_utc', 'domain', 'hasImage', 'id', 'image_url',
       'linked_submission_id', 'num_comments', 'score', 'subreddit', 'title_x',
       'upvote_ratio', '2_way_label', '3_way_label', '6_way_label',
       'reddit_url', 'external_url', 'is_link_post', 'has_valid_url',
       'scraped', 'article_name', 'url', 'title_y', 'text', 'publish_date',
       'authors', 'top_image', 'images', 'source', 'image_path'],
      dtype='object')


2_way_label
1    3944
0    3479
Name: count, dtype: int64

In [ ]:
fakeddit_df['label'] = 1 - fakeddit_df['label']
fakeddit_df["image_path"] = fakeddit_df['image_path'].str.replace("image_dump\\", "", regex=False)

In [ ]:
fakeddit_df.rename(columns={
    "title_y": "title",
    "2_way_label": "label",
}, inplace=True)

In [ ]:
fakeddit_cols = ["domain", "title_y", "text", "publish_date", "image_path", "score"]


## Weibo

In [10]:
weibo_df = pd.read_csv("../data/weibo/weibo_df.csv")

In [19]:
display(weibo_df)
print(weibo_df.columns)
weibo_df['label'].value_counts()

,tweet_id,user_name,tweet_url,user_url,publish_time,original,retweet_count,comment_count,praise_count,user_id,...,user_fans_count,user_follow_count,user_tweet_count,publish_platform,image_urls,tweet_content,label,valid_image,title,domain
0,3511947309647762,地球超级爆料,http://weibo.com/2803550292/z50NQEtP4,NaN,2012-11-13 16:55,True,79,18,0,2.803550e+09,...,5047.0,1770.0,1979.0,微博 weibo.com,['http://ww1.sinaimg.cn/large/a71ac854gw1dytin...,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,1,a71ac854gw1dytin2zmk9j.jpg,,weibo.com
1,3576100079039606,Noodles_Liu,http://weibo.com/1000432103/zvVI3BVC6,NaN,2013-05-09 17:36,True,1,0,0,1.000432e+09,...,9049.0,490.0,28017.0,iPhone客户端,['http://ww1.sinaimg.cn/large/3ba161e7jw1e4i6j...,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,1,3ba161e7jw1e4i6j4ep61j211n0r4juz.jpg,,weibo.com
2,3899073935617462,淡然一夏02,http://weibo.com/5649210127/CFzmFnzme,NaN,2015-10-17 23:18,True,0,0,0,5.649210e+09,...,8.0,82.0,378.0,vivo X5Pro,['http://ww1.sinaimg.cn/large/006ajvCngw1ex4jj...,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",1,006ajvCngw1ex4jjl2pwlj30c8096wf7.jpg,,weibo.com
3,3584521306131914,凤凰吴氏制茶,http://weibo.com/1628620650/zzsMGpJbQ,NaN,2013-06-01 23:19,True,5,16,0,1.628621e+09,...,3362.0,1908.0,5447.0,Weico.Android,['http://ww1.sinaimg.cn/large/6112c76ajw1e591p...,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",1,6112c76ajw1e591p39mpej20hw094mxh.jpg,,weibo.com
4,3553661986467439,咩咩百分百,http://weibo.com/2772451683/zmvZAr8tx,NaN,2013-03-08 19:35,True,0,0,0,2.772452e+09,...,32.0,20.0,133.0,三星Galaxy Note II,['http://ww3.sinaimg.cn/large/a5404163jw1e2ilj...,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,1,a5404163jw1e2iljwbrztj.jpg,,weibo.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,3914538635795904,NaN,NaN,NaN,1448782202000,True,233,27,362,NaN,...,NaN,NaN,NaN,中国新闻周刊网,['http://ww2.sinaimg.cn/large/61e6c012jw1eyhvn...,#下午茶#孩子心中的英雄梦！嗯嗯，就是酱滴 图 | Jason Ratliff,0,61e6c012jw1eyhvnvwfcwj20c80ict91.jpg,,weibo.com
6198,3914176248092770,NaN,NaN,NaN,1448695802000,True,369,7,399,NaN,...,NaN,NaN,NaN,中国新闻周刊网,['http://ww1.sinaimg.cn/large/61e6c012jw1eygq1...,#下午茶#心怀一颗淡然的心，才能看见多彩的世界。图丨Erin Hanson。,0,61e6c012jw1eygq1lec8pj20c808kwfw.jpg,,weibo.com
6199,3913901387088888,NaN,NaN,NaN,1448630270000,True,607,562,3900,NaN,...,NaN,NaN,NaN,iPhone客户端,['http://ww3.sinaimg.cn/large/473ed7c0jw1eyfud...,感谢《中国新闻周刊》，把这奖項颁给我！我衷心希望尘肺农民的生存困境能被人所知，也希望更多企业...,0,473ed7c0jw1eyfu91ujsuj2074074t9b.jpg,,weibo.com
6200,3913912015592182,NaN,NaN,NaN,1448632804000,True,2025,144,909,NaN,...,NaN,NaN,NaN,中国新闻周刊网,['http://ww2.sinaimg.cn/large/61e6c012jw1eyfvp...,#充电#【公务员考试常识600题】这才叫上知天文，下知地理！当然，即使不考公务员，这些基本常...,0,61e6c012jw1eyfvp23dtaj20c82tdgx0.jpg,,weibo.com


Index(['tweet_id', 'user_name', 'tweet_url', 'user_url', 'publish_time',
       'original', 'retweet_count', 'comment_count', 'praise_count', 'user_id',
       'user_auth_type', 'user_fans_count', 'user_follow_count',
       'user_tweet_count', 'publish_platform', 'image_urls', 'tweet_content',
       'label', 'valid_image', 'title', 'domain'],
      dtype='object')


label
1    3395
0    2807
Name: count, dtype: int64

In [13]:
weibo_df["title"] = ""
weibo_df["domain"] = "weibo.com"

In [ ]:
weibo_cols = ["domain", "title", "tweet_content", "publish_time", "valid_image", "praise_count"]

In [ ]:
display(weibo_df)
display(fakeddit_df)

,tweet_id,user_name,tweet_url,user_url,publish_time,original,retweet_count,comment_count,praise_count,user_id,...,user_fans_count,user_follow_count,user_tweet_count,publish_platform,image_urls,tweet_content,label,valid_image,title,domain
0,3511947309647762,地球超级爆料,http://weibo.com/2803550292/z50NQEtP4,null,2012-11-13 16:55,true,79,18,0,2803550292,...,5047,1770,1979,微博 weibo.com,[http://ww1.sinaimg.cn/large/a71ac854gw1dytin2...,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,1,a71ac854gw1dytin2zmk9j.jpg,,weibo.com
1,3576100079039606,Noodles_Liu,http://weibo.com/1000432103/zvVI3BVC6,null,2013-05-09 17:36,true,1,0,0,1000432103,...,9049,490,28017,iPhone客户端,[http://ww1.sinaimg.cn/large/3ba161e7jw1e4i6j4...,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,1,3ba161e7jw1e4i6j4ep61j211n0r4juz.jpg,,weibo.com
2,3899073935617462,淡然一夏02,http://weibo.com/5649210127/CFzmFnzme,null,2015-10-17 23:18,true,0,0,0,5649210127,...,8,82,378,vivo X5Pro,[http://ww1.sinaimg.cn/large/006ajvCngw1ex4jjl...,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",1,006ajvCngw1ex4jjl2pwlj30c8096wf7.jpg,,weibo.com
3,3584521306131914,凤凰吴氏制茶,http://weibo.com/1628620650/zzsMGpJbQ,null,2013-06-01 23:19,true,5,16,0,1628620650,...,3362,1908,5447,Weico.Android,[http://ww1.sinaimg.cn/large/6112c76ajw1e591p3...,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",1,6112c76ajw1e591p39mpej20hw094mxh.jpg,,weibo.com
4,3553661986467439,咩咩百分百,http://weibo.com/2772451683/zmvZAr8tx,null,2013-03-08 19:35,true,0,0,0,2772451683,...,32,20,133,三星Galaxy Note II,[http://ww3.sinaimg.cn/large/a5404163jw1e2iljw...,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,1,a5404163jw1e2iljwbrztj.jpg,,weibo.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7526,3914538635795904,null,null,null,1448782202000,true,233,27,362,null,...,null,null,null,中国新闻周刊网,[http://ww2.sinaimg.cn/large/61e6c012jw1eyhvnv...,#下午茶#孩子心中的英雄梦！嗯嗯，就是酱滴 图 | Jason Ratliff,0,61e6c012jw1eyhvnvwfcwj20c80ict91.jpg,,weibo.com
7527,3914176248092770,null,null,null,1448695802000,true,369,7,399,null,...,null,null,null,中国新闻周刊网,[http://ww1.sinaimg.cn/large/61e6c012jw1eygq1k...,#下午茶#心怀一颗淡然的心，才能看见多彩的世界。图丨Erin Hanson。,0,61e6c012jw1eygq1lec8pj20c808kwfw.jpg,,weibo.com
7528,3913901387088888,null,null,null,1448630270000,true,607,562,3900,null,...,null,null,null,iPhone客户端,[http://ww3.sinaimg.cn/large/473ed7c0jw1eyfudo...,感谢《中国新闻周刊》，把这奖項颁给我！我衷心希望尘肺农民的生存困境能被人所知，也希望更多企业...,0,473ed7c0jw1eyfu91ujsuj2074074t9b.jpg,,weibo.com
7529,3913912015592182,null,null,null,1448632804000,true,2025,144,909,null,...,null,null,null,中国新闻周刊网,[http://ww2.sinaimg.cn/large/61e6c012jw1eyfvp1...,#充电#【公务员考试常识600题】这才叫上知天文，下知地理！当然，即使不考公务员，这些基本常...,0,61e6c012jw1eyfvp23dtaj20c82tdgx0.jpg,,weibo.com


,Unnamed: 0.1,Unnamed: 0,index,author,clean_title,created_utc,domain,hasImage,id,image_url,...,article_name,url,title_y,text,publish_date,authors,top_image,images,source,image_path
0,0,5576,12354,residentchubbychaser,there are no good options in syria sighs man w...,1492109533,theonion.com,True,657h33,https://external-preview.redd.it/QGOLuH4_e7M5f...,...,usanews_theonion_scraped_articles_12354.json,https://www.texastribune.org/2016/10/28/uttt-p...,UT/TT Poll: Texas voters feeling high anxiety ...,Voters in the party that has not lost a statew...,2016-10-28,"['Ross Ramsey', 'Oct.', 'Am Central']",https://thumbnails.texastribune.org/YS7n_7vGhg...,['https://thumbnails.texastribune.org/YS7n_7vG...,https://www.texastribune.org,image_dump\TT-RossCharts_FRI-01.jpeg
1,1,3962,8782,dwaxe,not very scary honestly this little girl is po...,1568059431,clickhole.com,True,d1w7ur,https://external-preview.redd.it/nkiyXncogSDx5...,...,usanews_theonion_scraped_articles_8782.json,http://www.nj.com/politics/index.ssf/2017/11/m...,Phil Murphy beats Kim Guadagno to succeed Chri...,"TRENTON -- Phil Murphy, a former Wall Street e...",2017-11-08,"['Brent Johnson', 'Bjohnson Njadvancemedia.Com...",http://www.nj.com/pf/resources/images/nj/favic...,['http://www.nj.com/pf/resources/images/common...,http://www.nj.com,image_dump\favicon.ico
2,2,5841,12927,residentchubbychaser,god deploys more mosquitoes to us,1497908290,theonion.com,True,6i9l51,https://external-preview.redd.it/mjKGy8mJI-JO4...,...,usanews_theonion_scraped_articles_12927.json,https://www.clickhole.com/devastating-blow-to-...,Devastating Blow To Feminism: This Woman Sucks,Devastating Blow To Feminism: This Woman Sucks...,2019-06-12,"['Clickhole Editorial', 'June']",https://clickhole.com/wp-content/uploads/2019/...,['https://clickhole.com/wp-content/uploads/202...,https://www.clickhole.com,image_dump\sng0fvtlhsq8btayt79t.jpg
3,3,357,801,dwaxe,gordon ramsay said what,1553191320,clickhole.com,True,b3tree,https://external-preview.redd.it/5W3TOi9qCR1Ko...,...,usanews_theonion_scraped_articles_801.json,http://www.theverge.com/2017/1/29/14430082/fou...,"In the last 24 hours, four federal courts have...","Last night, a Federal District court issued an...",2017-01-29,['Andrew Liptak'],https://platform.theverge.com/wp-content/uploa...,['https://platform.theverge.com/wp-content/upl...,http://www.theverge.com,image_dump\DSC_0178.0.jpg
4,4,1804,4087,dwaxe,blog i wont let my son play wheelchair basketb...,1505053996,clickhole.com,True,6z8j5j,https://external-preview.redd.it/vJiR9pAFoQSTV...,...,usanews_theonion_scraped_articles_4087.json,http://www.foxnews.com/health/2016/11/04/healt...,Health officials find first cases of new super...,Just five months after federal health official...,2016-11-04,[],https://static.foxnews.com/foxnews.com/content...,['https://a57.foxnews.com/cf-images.us-east-1....,http://www.foxnews.com,image_dump\candida-auris_2016.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7418,8082,21816,32140,Seitz_,brexit mps vote by to to seek delay to eu depa...,1552590210,bbc.co.uk,True,b14akb,https://external-preview.redd.it/C3jff__SjprvR...,...,neutralnews_nottheonion_scraped_articles_32140...,http://www.bbc.co.uk/news/uk-politics-47576813,Brexit: MPs vote by a majority of 211 to seek ...,MPs have voted by 413 to 202 - a majority of 2...,NaN,[],https://ichef.bbci.co.uk/ace/branded_news/1200...,['https://ichef.bbci.co.uk/ace/standard/385/cp...,http://www.bbc.co.uk,image_dump\_106029989_ukparliament_jessicatayl...
7419,8083,1622,2257,limahart,field hockey team rallies around teammate with...,1474724364,boston.cbslocal.com,True,54a5kh,https://external-preview.redd.it/VYkVPVWBJfGEc...,...,upliftingnews_usnews_scraped_articles_2257.json,http://boston.cbslocal.com/2016/09/23/tewksbur...,Field Hockey Team Rallies Around Player Living...,TEWKSBURY (CBS) - It's a story that plays out ...,2016-09-23,[],https://assets1.cbsnewsstatic.com/hub/i/r/2015...,['https:/

In [ ]:
weibo_df.rename(columns={
    "tweet_content": "text",
    "publish_time": "publish_date",
    "valid_image": "image_path",
    "praise_count": "score",
}, inplace=True)

## FakeNewsNet

In [ ]:
fakenews_df = pd.read_csv("../data/fakenewsnet/fakenewsnet_processed4.csv")

In [117]:
fakenews_df

,domain,title,text,publish_date,image_path,score,label
0,yournewswire.com,Bill Gates’ Former Doctor Says Billionaire ‘Re...,Massachusetts Urged to Pass Sports Betting Bil...,2022-06-01 09:18:59+00:00,FakeNewsNet//images/politifact15342.jpe,0,1
1,web.archive.org,TRUMP WANTS TO DEPORT AMERICAN INDIANS TO INDIA,Washington (dpo) – As part of his plan to impr...,2017-04-25 00:00:00,FakeNewsNet//images/politifact14043.jpg,0,1
2,beforeitsnews.com,Target to Discontinue Sale of Holy Bible,\n\n\n\nTarget CEO Brian Cornell announced tod...,2016-05-18 05:57:35,FakeNewsNet//images/politifact13775.jpg,0,1
3,www.wilmingtonfilm.com,Wilmington Regional Film Commission,LOCATION. LOCATION. LOCATION.\n\nSince the 198...,NaN,FakeNewsNet//images/politifact13943.ico,0,1
4,actionnews3.com,"Legendary Actor Kirk Douglas Dead, 4 Days Befo...",News reports have confirmed that actor Kirk Do...,2017-12-06 03:56:22+00:00,FakeNewsNet//images/politifact15095.jpg,0,1
...,...,...,...,...,...,...,...
1074,www.thrillist.com,"The 13 Most Tear-Jerking 'This Is Us' Moments,...",NBC NBC\n\nThis Is Us wrapped up its powerful ...,2018-03-14 21:33:52+00:00,FakeNewsNet//images/gossipcop-836336.jpg,0,0
1075,deadline.com,Lin-Manuel Miranda & James Corden Strip Down T...,Lin-Manuel Miranda and James Corden shed their...,2017-08-18 17:04:16+00:00,FakeNewsNet//images/gossipcop-874392.jpg,0,0
1076,www.thewrap.com,"Julia Louis-Dreyfus Thanks Katy Perry, Wraps 2...",Julia Louis-Dreyfus just wrapped her second ro...,2017-10-20 14:25:41+00:00,FakeNewsNet//images/gossipcop-888391.jpg,0,0
1077,www.thewrap.com,Jimmy Kimmel Learns Secret of the ‘Whaboom’ Fr...,(Spoiler alert: Please do not read on if you h...,2017-06-06 05:21:09+00:00,FakeNewsNet//images/gossipcop-859228.jpg,0,0


In [ ]:
clean_invalid_images('image_dump')


Deleting invalid image: politifact211.png (cannot identify image file 'image_dump/politifact211.png')
Deleting invalid image: gossipcop-955939.jpg (cannot identify image file 'image_dump/gossipcop-955939.jpg')
Deleting invalid image: politifact14621.jpg (cannot identify image file 'image_dump/politifact14621.jpg')
Deleting invalid image: politifact13600.jpg (cannot identify image file 'image_dump/politifact13600.jpg')
Deleting invalid image: gossipcop-869145.png (cannot identify image file 'image_dump/gossipcop-869145.png')
Deleting invalid image: politifact8005.png (cannot identify image file 'image_dump/politifact8005.png')
Deleting invalid image: gossipcop-896408.png (cannot identify image file 'image_dump/gossipcop-896408.png')
Deleting invalid image: politifact14596.jpg (cannot identify image file 'image_dump/politifact14596.jpg')
Deleting invalid image: gossipcop-914542.jpg (cannot identify image file 'image_dump/gossipcop-914542.jpg')
Deleting invalid image: politifact14187.jpg 

In [118]:
fakenews_df["image_path"] = fakenews_df["image_path"].apply(lambda x: x.split("/")[-1])

In [119]:
fakenews_df = fakenews_df[fakenews_df['image_path'].apply(
    lambda image_filename: os.path.exists(os.path.join(source_folder, image_filename)) if image_filename else False
)] 

In [120]:
fakenews_df

,domain,title,text,publish_date,image_path,score,label
0,yournewswire.com,Bill Gates’ Former Doctor Says Billionaire ‘Re...,Massachusetts Urged to Pass Sports Betting Bil...,2022-06-01 09:18:59+00:00,politifact15342.jpe,0,1
2,beforeitsnews.com,Target to Discontinue Sale of Holy Bible,\n\n\n\nTarget CEO Brian Cornell announced tod...,2016-05-18 05:57:35,politifact13775.jpg,0,1
3,www.wilmingtonfilm.com,Wilmington Regional Film Commission,LOCATION. LOCATION. LOCATION.\n\nSince the 198...,NaN,politifact13943.ico,0,1
4,actionnews3.com,"Legendary Actor Kirk Douglas Dead, 4 Days Befo...",News reports have confirmed that actor Kirk Do...,2017-12-06 03:56:22+00:00,politifact15095.jpg,0,1
5,yournewswire.com,Computer Models Show IRMA Destroying New York ...,Take a look at our list below of the best New ...,2022-04-01 08:38:53+00:00,politifact14503.jpe,0,1
...,...,...,...,...,...,...,...
1074,www.thrillist.com,"The 13 Most Tear-Jerking 'This Is Us' Moments,...",NBC NBC\n\nThis Is Us wrapped up its powerful ...,2018-03-14 21:33:52+00:00,gossipcop-836336.jpg,0,0
1075,deadline.com,Lin-Manuel Miranda & James Corden Strip Down T...,Lin-Manuel Miranda and James Corden shed their...,2017-08-18 17:04:16+00:00,gossipcop-874392.jpg,0,0
1076,www.thewrap.com,"Julia Louis-Dreyfus Thanks Katy Perry, Wraps 2...",Julia Louis-Dreyfus just wrapped her second ro...,2017-10-20 14:25:41+00:00,gossipcop-888391.jpg,0,0
1077,www.thewrap.com,Jimmy Kimmel Learns Secret of the ‘Whaboom’ Fr...,(Spoiler alert: Please do not read on if you h...,2017-06-06 05:21:09+00:00,gossipcop-859228.jpg,0,0


## Joining the 3 datasets

In [ ]:
merged = pd.concat([weibo_df[finals_cols], fakeddit_df[finals_cols]], ignore_index=True)
merge = pd.concat([merged, fakenews_df], ignore_index=True)

In [ ]:
merged["image_exists"].value_counts()

image_exists
True     13624
False        1
Name: count, dtype: int64

In [180]:
merge['label'].value_counts()

label
0    7623
1    6978
Name: count, dtype: int64

In [ ]:
# Define the source folder where images are currently stored
source_folder = 'image_dump/'  # Folder where images are currently stored

if not os.path.exists(source_folder):
    print(f"The source folder {source_folder} does not exist!")
    exit()

# Assuming your dataframe is named `weibo_df` and the 'valid_image' column contains the image names/paths
merge['image_exists'] = merge['image_path'].apply(
    lambda image_filename: os.path.exists(os.path.join(source_folder, image_filename)) if image_filename else False
)
merge["image_exists"].value_counts()

image_exists
True     14600
False        1
Name: count, dtype: int64

In [ ]:
merge[final_cols].to_csv("full_train.csv", index=False)